In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('..')

In [3]:
from Bio import SeqIO

In [4]:
import re
import random
import numpy as np
import pandas as pd
from itertools import product
from mofiwo.utility import generate_utr_from_cdna_cds, load_rna_fasta_zipfile
from mofiwo.analysis import generate_k_mer, generate_feature_by_kmer_loc

In [18]:
from sklearn import linear_model
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [6]:
_folder = r'C:\Users\by003457\workspace\rnamotif\samples'

In [7]:
zip_cdna = os.path.join(_folder,'s1_cdna.zip')
zip_cds = os.path.join(_folder,'s1_cds.zip')

In [8]:
dic_cdna = load_rna_fasta_zipfile(zip_cdna)
dic_cds = load_rna_fasta_zipfile(zip_cds)

KeyboardInterrupt: 

In [ ]:
dic_utr = generate_utr_from_cdna_cds(dic_cdna, dic_cds)

In [22]:
dic_seq = dict()
for k,v in dic_utr.items():
    dic_seq.update({k: {'CDS': dic_cds[k].seq, 'UT3': v['utr3'], 'UT5': v['utr5']}})

In [23]:
ret = generate_feature_by_kmer_loc(dic_seq, need_log=True)

# proceed: 100
# proceed: 200
# proceed: 300
# proceed: 400
# proceed: 500
# proceed: 600
# proceed: 700
# proceed: 800
# proceed: 900
# proceed: 1000
# proceed: 1100
# proceed: 1200
# proceed: 1300
# proceed: 1400
# proceed: 1500
# proceed: 1600
# proceed: 1700
# proceed: 1800
# proceed: 1900
# proceed: 2000
# proceed: 2100
# proceed: 2200
# proceed: 2300
# proceed: 2400
# proceed: 2500
# proceed: 2600
# proceed: 2700
# proceed: 2800
# proceed: 2900
# proceed: 3000
# proceed: 3100
# proceed: 3200
# proceed: 3300
# proceed: 3400
# proceed: 3500
# proceed: 3600
# proceed: 3700
# proceed: 3800
# proceed: 3900
# proceed: 4000
# proceed: 4100
# proceed: 4200
# proceed: 4300
# proceed: 4400
# proceed: 4500
# proceed: 4600
# proceed: 4700
# proceed: 4800
# proceed: 4900
# proceed: 5000
# proceed: 5100
# proceed: 5200
# proceed: 5300
# proceed: 5400
# proceed: 5500
# proceed: 5600
# proceed: 5700
# proceed: 5800
# proceed: 5900
# proceed: 6000
# proceed: 6100
# proceed: 6200
# proceed: 6300
#

In [24]:
df = pd.DataFrame(ret)

In [25]:
df.shape

(4032, 7252)

In [26]:
df.to_parquet('./notebook/feature_intestine.gzip.parquet',compression='gzip')

In [9]:
df= pd.read_parquet('./notebook/feature_intestine.gzip.parquet')

In [10]:
df.fillna(0.0,inplace=True)

In [11]:
df_s1 = pd.read_excel(os.path.join(_folder,'aan2399_table_S1.xlsx'))

In [31]:
df_s1 = df_s1[df_s1.target_id.isin(list(df.columns))]
df_s1['b2'] = df_s1.b.apply(lambda x: 1 if x>0 else -1)

In [32]:
cols = list(df_s1[(abs(df_s1.b2) < 4) & (df_s1.mean_obs > 3.5) & (df_s1.qval < 0.2)].target_id)
len(cols)

475

In [33]:
df_x = df[cols]
df_y = {x: int(df_s1[df_s1.target_id == x]['b2']) for x in cols}
df_y = pd.DataFrame(df_y, index=[0])

In [34]:
df_x2 = df_x.T
df_y2 = df_y.T
df_y2.columns = ['Class']
df_xy = pd.merge(df_x2,df_y2, left_index=True, right_index=True)

In [35]:
X = df_xy.iloc[:,:-1].values
y = df_xy.iloc[:,-1].values

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [37]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [38]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import NearestNeighbors

regressor = RandomForestClassifier(n_estimators=1000, random_state=0)
# regressor = NearestNeighbors(n_neighbors=100, algorithm='ball_tree')
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [39]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[ 2 19]
 [ 0 74]]
              precision    recall  f1-score   support

          -1       1.00      0.10      0.17        21
           1       0.80      1.00      0.89        74

    accuracy                           0.80        95
   macro avg       0.90      0.55      0.53        95
weighted avg       0.84      0.80      0.73        95

0.8
